In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random


from tableone import TableOne

pd.set_option('display.max_rows', 200)     # Display max 200 rows in a pandas dataframe
pd.set_option('display.max_columns', None) # Display all columns in a pandas dataframe

In [2]:
# Load data
data = pd.read_csv("data/final/data_imputed.csv", parse_dates=["date"])
data = data[data["postmenopausal"] == 1]
data

,centerId,patientId,date,postmenopausal,praemenopausal,hrt,height,weight,bmi,hip_fracture_parents,osteoporotic_fracture_parents,corticosteroids,steroid_daily_dosage,prostate_cancer,aromatase_inhibitors,antiepileptic_drugs,rheumatoid_arthritis,ankylosing_spondylitis,number_of_falls,immobility,type_1_diabetes,copd,gastrointestinal_disease,hypogonadismus_mann,early_menopause,hyperpara,malfunction_of_kidney,alcohol,nicotin,decrease_in_height,low_back_pain,hyperkyphosis,falling_test_abnormal,hiv positiv,deathdate,death_by_osteoporosis,sex,age,Bisphosphonat_prior,SERM_prior,HRT_prior,Denosumab_prior,Romosozumab_prior,Teriparatide_prior,Bisphosphonat_current,SERM_current,HRT_current,Denosumab_current,Romosozumab_current,Teriparatide_current,Bisphosphonat_new,SERM_new,HRT_new,Denosumab_new,Romosozumab_new,Teriparatide_new,No_treatment,vertebral_fracture,hip_fracture,wrist_fracture,any_fracture,vertebral_right_censored,hip_right_censored,wrist_right_censored,any_right_censored,recent_fracture,previous_fracture,num_visits,tscore_ls,tscore_neck,tscore_totalHip,tbs_ls,tscore_ls_imputed,tscore_neck_imputed,tscore_totalHip_imputed,tbs_ls_imputed
0,8,1,2018-06-01,1.0,0.0,0.0,156.5,59.5,24.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,73.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,28.689,28.689,28.689,28.689,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-1.0,-1.8,-1.2,1.470,0.0,0.0,0.0,0.0
1,8,3,2018-06-13,1.0,0.0,0.0,153.0,68.1,29.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,female,80.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,27.574,27.574,27.574,27.574,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-2.6,-2.3,-2.0,1.169,0.0,0.0,0.0,0.0
2,8,4,2018-06-22,1.0,0.0,0.0,165.5,73.2,26.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,69.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,28.164,28.164,28.164,28.164,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-3.8,-3.2,-3.1,1.290,0.0,0.0,0.0,0.0
3,8,5,2018-06-22,1.0,0.0,0.0,150.0,40.9,18.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,76.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,28.197,28.197,28.197,6.328,1.0,1.0,1.0,0.0,3.0,3.0,1.0,-3.1,-4.2,-3.3,1.100,1.0,0.0,0.0,1.0
4,8,6,2018-06-27,1.0,0.0,0.0,165.5,74.7,27.3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,59.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,25.279,25.279,25.279,25.279,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-2.5,-1.3,-0.9,1.295,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11621,2,19583,2022-04-19,1.0,0.0,0.0,149.0,68.0,30.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,84.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.033,0.033,0.033,0.033,0.0,1.0,1.0,0.0,2.0,2.0,1.0,-2.0,-2.9,-3.1,1.166,0.0,0.0,0.0,0.0
11622,2,19590,2020-01-15,1.0,0.0,0.0,151.0,79.0,34.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,82.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,27.049,27.049,27.049,27.049,1.0,1.0,1.0,1.0,1.0,3.0,1.0,-2.5,-2.8,-2.1,1.075,0.0,0.0,0.0,0.0
11623,2,19654,2020-02-13,1.0,0.0,0.0,156.5,73.0,29.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,61.1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,26.295,26.295,26.295,26.295,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-2.8,-1.4,-0.5,1.340,0.0,0.0,0.0,0.0
11624,2,19708,2014-09-17,1.0,0.0,1.0,168.0,71.0,25.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.

In [3]:
def get_x_year_fractures(visit, years=2):
    fracture_columns = ["vertebral_fracture", "hip_fracture", "any_fracture"]
    for fracture in fracture_columns:
        fracture_type = fracture.split("_")[0]
        if visit[f"{fracture_type}_right_censored"] == 0 and visit[fracture] <= (years*12):
            visit[fracture] = 1
        else:
            visit[fracture] = 0
    return visit

In [4]:
data = data.apply(lambda x: get_x_year_fractures(x, years=100), axis=1)
data = data.sort_values(["patientId", "date"])
data = data.groupby("patientId").first().reset_index()
data.dropna(inplace=True)
data

,patientId,centerId,date,postmenopausal,praemenopausal,hrt,height,weight,bmi,hip_fracture_parents,osteoporotic_fracture_parents,corticosteroids,steroid_daily_dosage,prostate_cancer,aromatase_inhibitors,antiepileptic_drugs,rheumatoid_arthritis,ankylosing_spondylitis,number_of_falls,immobility,type_1_diabetes,copd,gastrointestinal_disease,hypogonadismus_mann,early_menopause,hyperpara,malfunction_of_kidney,alcohol,nicotin,decrease_in_height,low_back_pain,hyperkyphosis,falling_test_abnormal,hiv positiv,deathdate,death_by_osteoporosis,sex,age,Bisphosphonat_prior,SERM_prior,HRT_prior,Denosumab_prior,Romosozumab_prior,Teriparatide_prior,Bisphosphonat_current,SERM_current,HRT_current,Denosumab_current,Romosozumab_current,Teriparatide_current,Bisphosphonat_new,SERM_new,HRT_new,Denosumab_new,Romosozumab_new,Teriparatide_new,No_treatment,vertebral_fracture,hip_fracture,wrist_fracture,any_fracture,vertebral_right_censored,hip_right_censored,wrist_right_censored,any_right_censored,recent_fracture,previous_fracture,num_visits,tscore_ls,tscore_neck,tscore_totalHip,tbs_ls,tscore_ls_imputed,tscore_neck_imputed,tscore_totalHip_imputed,tbs_ls_imputed
0,1,8,2018-06-01,1.0,0.0,0.0,156.5,59.5,24.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,73.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.689,0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-1.0,-1.8,-1.2,1.470,0.0,0.0,0.0,0.0
1,3,8,2018-06-13,1.0,0.0,0.0,153.0,68.1,29.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,female,80.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,27.574,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-2.6,-2.3,-2.0,1.169,0.0,0.0,0.0,0.0
2,4,8,2018-06-22,1.0,0.0,0.0,165.5,73.2,26.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,69.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.164,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-3.8,-3.2,-3.1,1.290,0.0,0.0,0.0,0.0
3,5,8,2018-06-22,1.0,0.0,0.0,150.0,40.9,18.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,76.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.197,1,1.0,1.0,1.0,0.0,3.0,3.0,1.0,-3.1,-4.2,-3.3,1.100,1.0,0.0,0.0,1.0
4,6,8,2018-06-27,1.0,0.0,0.0,165.5,74.7,27.3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,59.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,25.279,0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-2.5,-1.3,-0.9,1.295,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,19583,2,2022-04-19,1.0,0.0,0.0,149.0,68.0,30.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,84.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0.033,1,0.0,1.0,1.0,0.0,2.0,2.0,1.0,-2.0,-2.9,-3.1,1.166,0.0,0.0,0.0,0.0
6171,19590,2,2020-01-15,1.0,0.0,0.0,151.0,79.0,34.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,82.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,27.049,0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,-2.5,-2.8,-2.1,1.075,0.0,0.0,0.0,0.0
6172,19654,2,2020-02-13,1.0,0.0,0.0,156.5,73.0,29.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,61.1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,26.295,0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-2.8,-1.4,-0.5,1.340,0.0,0.0,0.0,0.0
6173,19708,2,2014-09-17,1.0,0.0,1.0,168.0,71.0,25.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,45.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,91.180,0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,-0.5,-1.6,-1.0,1.168,0.

In [5]:
data[["vertebral_fracture", "hip_fracture", "any_fracture"]].sum()

vertebral_fracture     449
hip_fracture           177
any_fracture          1190
dtype: int64

In [6]:
data["osteoporosis"] = (data[["tscore_ls", "tscore_neck", "tscore_totalHip"]].min(axis=1) <= -2.5)
data["osteoporosis"].sum()

2845

In [7]:
binary_columns = [
    "recent_fracture", 
    "previous_fracture", 
    "Bisphosphonat_prior",
    "SERM_prior",
    "HRT_prior",
    "Denosumab_prior",
    "Romosozumab_prior",
    "Teriparatide_prior",
    "Bisphosphonat_current",
    "SERM_current",
    "HRT_current",
    "Denosumab_current",
    "Romosozumab_current",
    "Teriparatide_current",
    "Bisphosphonat_new",
    "SERM_new",
    "HRT_new",
    "Denosumab_new",
    "Romosozumab_new",
    "Teriparatide_new",
    "No_treatment",
    "osteoporosis",
]
for col in binary_columns:
    data.loc[:, col] = data.loc[:,col].apply(lambda x: int(bool(x)))
    print(col)
    print(data[col].value_counts())

recent_fracture
0    4308
1    1636
Name: recent_fracture, dtype: int64
previous_fracture
0    3264
1    2680
Name: previous_fracture, dtype: int64
Bisphosphonat_prior
0    4813
1    1131
Name: Bisphosphonat_prior, dtype: int64
SERM_prior
0    5881
1      63
Name: SERM_prior, dtype: int64
HRT_prior
0    5926
1      18
Name: HRT_prior, dtype: int64
Denosumab_prior
0    5687
1     257
Name: Denosumab_prior, dtype: int64
Romosozumab_prior
0    5944
Name: Romosozumab_prior, dtype: int64
Teriparatide_prior
0    5846
1      98
Name: Teriparatide_prior, dtype: int64
Bisphosphonat_current
0    5238
1     706
Name: Bisphosphonat_current, dtype: int64
SERM_current
0    5899
1      45
Name: SERM_current, dtype: int64
HRT_current
0    5858
1      86
Name: HRT_current, dtype: int64
Denosumab_current
0    5728
1     216
Name: Denosumab_current, dtype: int64
Romosozumab_current
0    5943
1       1
Name: Romosozumab_current, dtype: int64
Teriparatide_current
0    5912
1      32
Name: Teriparatide_curr

In [8]:
# add random noise to the "steroid daily dosage" and "number of falls" columns.
# this is necessary for the tableone package to recognise these columns as continuous
data["steroid_daily_dosage"] = data["steroid_daily_dosage"].apply(lambda x: x + np.exp(random.uniform(-10, -5)))
data["number_of_falls"] = data["number_of_falls"].apply(lambda x: x + np.exp(random.uniform(-10, -5)))

In [10]:
data

,patientId,centerId,date,postmenopausal,praemenopausal,hrt,height,weight,bmi,hip_fracture_parents,osteoporotic_fracture_parents,corticosteroids,steroid_daily_dosage,prostate_cancer,aromatase_inhibitors,antiepileptic_drugs,rheumatoid_arthritis,ankylosing_spondylitis,number_of_falls,immobility,type_1_diabetes,copd,gastrointestinal_disease,hypogonadismus_mann,early_menopause,hyperpara,malfunction_of_kidney,alcohol,nicotin,decrease_in_height,low_back_pain,hyperkyphosis,falling_test_abnormal,hiv positiv,deathdate,death_by_osteoporosis,sex,age,Bisphosphonat_prior,SERM_prior,HRT_prior,Denosumab_prior,Romosozumab_prior,Teriparatide_prior,Bisphosphonat_current,SERM_current,HRT_current,Denosumab_current,Romosozumab_current,Teriparatide_current,Bisphosphonat_new,SERM_new,HRT_new,Denosumab_new,Romosozumab_new,Teriparatide_new,No_treatment,vertebral_fracture,hip_fracture,wrist_fracture,any_fracture,vertebral_right_censored,hip_right_censored,wrist_right_censored,any_right_censored,recent_fracture,previous_fracture,num_visits,tscore_ls,tscore_neck,tscore_totalHip,tbs_ls,tscore_ls_imputed,tscore_neck_imputed,tscore_totalHip_imputed,tbs_ls_imputed,osteoporosis
0,1,8,2018-06-01,1.0,0.0,0.0,156.5,59.5,24.3,0.0,0.0,0.0,0.000115,0.0,0.0,0.0,0.0,0.0,0.001889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,73.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.689,0,1.0,1.0,1.0,1.0,0,0,1.0,-1.0,-1.8,-1.2,1.470,0.0,0.0,0.0,0.0,0
1,3,8,2018-06-13,1.0,0.0,0.0,153.0,68.1,29.1,0.0,0.0,0.0,0.000550,0.0,0.0,0.0,0.0,0.0,1.000265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,female,80.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,27.574,0,1.0,1.0,1.0,1.0,1,1,1.0,-2.6,-2.3,-2.0,1.169,0.0,0.0,0.0,0.0,1
2,4,8,2018-06-22,1.0,0.0,0.0,165.5,73.2,26.7,0.0,0.0,0.0,0.000284,0.0,0.0,0.0,0.0,0.0,0.000623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,69.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.164,0,1.0,1.0,1.0,1.0,1,1,1.0,-3.8,-3.2,-3.1,1.290,0.0,0.0,0.0,0.0,1
3,5,8,2018-06-22,1.0,0.0,0.0,150.0,40.9,18.2,0.0,0.0,0.0,0.005343,0.0,0.0,0.0,0.0,0.0,3.002367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,76.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28.197,1,1.0,1.0,1.0,0.0,1,1,1.0,-3.1,-4.2,-3.3,1.100,1.0,0.0,0.0,1.0,1
4,6,8,2018-06-27,1.0,0.0,0.0,165.5,74.7,27.3,1.0,1.0,0.0,0.000490,0.0,0.0,0.0,0.0,0.0,0.000048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,59.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,25.279,0,1.0,1.0,1.0,1.0,0,0,1.0,-2.5,-1.3,-0.9,1.295,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6170,19583,2,2022-04-19,1.0,0.0,0.0,149.0,68.0,30.6,0.0,0.0,0.0,0.006046,0.0,0.0,0.0,0.0,0.0,0.000081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,84.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0.033,1,0.0,1.0,1.0,0.0,1,1,1.0,-2.0,-2.9,-3.1,1.166,0.0,0.0,0.0,0.0,1
6171,19590,2,2020-01-15,1.0,0.0,0.0,151.0,79.0,34.6,0.0,0.0,0.0,0.002000,0.0,0.0,0.0,0.0,0.0,0.000081,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,female,82.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,27.049,0,1.0,1.0,1.0,1.0,1,1,1.0,-2.5,-2.8,-2.1,1.075,0.0,0.0,0.0,0.0,1
6172,19654,2,2020-02-13,1.0,0.0,0.0,156.5,73.0,29.8,0.0,0.0,0.0,0.000156,0.0,0.0,0.0,0.0,0.0,0.001123,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,female,61.1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,26.295,0,1.0,1.0,1.0,1.0,0,0,1.0,-2.8,-1.4,-0.5,1.340,0.0,0.0,0.0,0.0,1
6173,19708,2,2014-09-17,1.0,0.0,1.0,168.0,71.0,25.2,0.0,0.0,0.0,0.003265,0.0,0.0,0.0,0.0,0.0,0.000296,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female,45.2,0,0,0,0,0,0

In [13]:
columns = data.columns.tolist()

# Drop non-relevant columns
columns = [col for col in columns if col not in [
    "patientId", "date", "centerId", "sex", "any_fracture", "prostate_cancer", "hypogonadismus_mann", "num_visits",
    "malfunction_of_kidney", "hiv positiv", "deathdate", "death_by_osteoporosis", "tscore_ls_imputed",
    "tscore_neck_imputed", "tscore_totalHip_imputed", "tbs_ls_imputed", "praemenopausal", "postmenopausal",
    "vertebral_right_censored", "hip_right_censored", "wrist_right_censored", "any_right_censored", "wrist_fracture",
    ]]

binary = [
    'hrt', 'hip_fracture_parents', 'osteoporotic_fracture_parents', 'corticosteroids', 'aromatase_inhibitors', 'antiepileptic_drugs',
    'rheumatoid_arthritis', 'ankylosing_spondylitis', 'immobility', 'type_1_diabetes', 'copd', 'gastrointestinal_disease', 'early_menopause',
    'hyperpara', 'alcohol', 'nicotin', 'decrease_in_height', 'low_back_pain', 'hyperkyphosis', 'falling_test_abnormal',
    'Bisphosphonat_prior', 'SERM_prior', 'HRT_prior', 'Denosumab_prior', 'Romosozumab_prior', 'Teriparatide_prior', 'Bisphosphonat_current',
    'SERM_current', 'HRT_current', 'Denosumab_current', 'Romosozumab_current', 'Teriparatide_current', 'Bisphosphonat_new', 'SERM_new', 'HRT_new', 'Denosumab_new',
    'Romosozumab_new', 'Teriparatide_new', 'No_treatment', 'vertebral_fracture', 'hip_fracture', 'recent_fracture', 'previous_fracture', 'osteoporosis'
 ]

table1 = TableOne(data, columns, categorical=binary, missing=False, dip_test=True, normal_test=True, tukey_test=True, groupby="any_fracture", pval=True)
table1

Grouped by any_fracture                                    
                                                         Overall             0             1 P-Value
n                                                           5944          4754          1190        
hrt, n (%)                           0.0             5654 (95.1)   4500 (94.7)   1154 (97.0)   0.001
                                     1.0               290 (4.9)     254 (5.3)      36 (3.0)        
height, mean (SD)                                    161.1 (6.4)   161.1 (6.4)   160.9 (6.5)   0.415
weight, mean (SD)                                    62.9 (12.5)   62.8 (12.7)   63.0 (12.0)   0.715
bmi, mean (SD)                                        24.3 (4.8)    24.2 (4.9)    24.3 (4.6)   0.496
hip_fracture_parents, n (%)          0.0             5291 (89.0)   4228 (88.9)   1063 (89.3)   0.738
                                     1.0              653 (11.0)    526 (11.1)    127 (10.7)        
osteoporotic_fracture_parents, n (%) 0.0             4992 (84.0)   4005 (84.2)    987 (82.9)   0.293
                                     1.0              952 (16.0)    749 (15.8)    203 (17.1)        
corticosteroids, n (%)               0.0             5514 (92.8)   4422 (93.0)   1092 (91.8)   0.153
                                     1.0               430 (7.2)     332 (7.0)      98 (8.2)        
steroid_daily_dosage, mean (SD)                        0.5 (2.9)     0.5 (3.0)     0.5 (2.4)   0.792
aromatase_inhibitors, n (%)          0.0             5606 (94.3)   4465 (93.9)   1141 (95.9)   0.011
                                     1.0               338 (5.7)     289 (6.1)      49 (4.1)        
antiepileptic_drugs, n (%)           0.0             5914 (99.5)   4732 (99.5)   1182 (99.3)   0.494
                                     1.0                30 (0.5)      22 (0.5)       8 (0.7)        
rheumatoid_arthritis, n (%)          0.0             5744 (96.6)   4606 (96.9)   1138 (95.6)   0.039
                                     1.0               200 (3.4)     148 (3.1)      52 (4.4)        
ankylosing_spondylitis, n (%)        0.0             5875 (98.8)   4697 (98.8)   1178 (99.0)   0.691
                                     1.0                69 (1.2)      57 (1.2)      12 (1.0)        
number_of_falls, mean (SD)                             0.1 (0.7)     0.1 (0.7)     0.1 (0.6)   0.027
immobility, n (%)                    0.0             5782 (97.3)   4634 (97.5)   1148 (96.5)   0.071
                                     1.0               162 (2.7)     120 (2.5)      42 (3.5)        
type_1_diabetes, n (%)               0.0             5903 (99.3)   4727 (99.4)   1176 (98.8)   0.038
                                     1.0                41 (0.7)      27 (0.6)      14 (1.2)        
copd, n (%)                          0.0             5817 (97.9)   4653 (97.9)   1164 (97.8)   0.987
                                     1.0               127 (2.1)     101 (2.1)      26 (2.2)        
gastrointestinal_disease, n (%)      0.0             5618 (94.5)   4479 (94.2)   1139 (95.7)   0.050
                                     1.0               326 (5.5)     275 (5.8)      51 (4.3)        
early_menopause, n (%)               0.0             5379 (90.5)   4290 (90.2)   1089 (91.5)   0.199
                                     1.0               565 (9.5)     464 (9.8)     101 (8.5)        
hyperpara, n (%)                     0.0             5866 (98.7)   4686 (98.6)   1180 (99.2)   0.145
                                     1.0                78 (1.3)      68 (1.4)      10 (0.8)        
alcohol, n (%)                       0.0             5867 (98.7)   4699 (98.8)   1168 (98.2)   0.081
                                     1.0                77 (1.3)      55 (1.2)      22 (1.8)        
nicotin, n (%)                       0.0             5443 (91.6)   4361 (91.7)   1082 (90.9)   0.401
                                     1.0               501 (8.4)     393 (8.3)     108 (9.1)        
d

Since it isn't clearly documented which statistical tests are used in the table one package, we are going to recalculate the p-values for the categorical variables using the Fisher's exact test and for the continuous variables using the Wilcoxon rank-sum test.

In [19]:
data = data.drop(columns=["wrist_fracture"])
categorical = [
    'hrt', 'hip_fracture_parents', 'osteoporotic_fracture_parents', 'corticosteroids', 'aromatase_inhibitors', 'antiepileptic_drugs',
    'rheumatoid_arthritis', 'ankylosing_spondylitis','immobility', 'type_1_diabetes', 'copd', 'gastrointestinal_disease', 'early_menopause',
    'hyperpara', 'alcohol', 'nicotin', 'decrease_in_height', 'low_back_pain', 'hyperkyphosis', 'falling_test_abnormal',
    'Bisphosphonat_prior', 'SERM_prior', 'HRT_prior', 'Denosumab_prior', 'Romosozumab_prior', 'Teriparatide_prior', 'Bisphosphonat_current',
    'SERM_current', 'HRT_current', 'Denosumab_current', 'Romosozumab_current', 'Teriparatide_current', 'Bisphosphonat_new', 'SERM_new', 'HRT_new', 'Denosumab_new',
    'Romosozumab_new', 'Teriparatide_new', 'No_treatment', 'vertebral_fracture', 'hip_fracture', 'recent_fracture', 'previous_fracture', 'osteoporosis'
 ]
continuous = [col for col in columns if col not in categorical]
continuous

['height',
 'weight',
 'bmi',
 'steroid_daily_dosage',
 'number_of_falls',
 'age',
 'tscore_ls',
 'tscore_neck',
 'tscore_totalHip',
 'tbs_ls']

In [20]:
from scipy.stats import ranksums

# Compute p-values for continuous variables with Wilcoxon rank-sum test
for col in continuous:
    print(col)
    no_fractures = data[data["any_fracture"] == 0][col]
    any_fractures = data[data["any_fracture"] == 1][col]
    print(round(ranksums(no_fractures, any_fractures).pvalue, 3))

height
0.508
weight
0.276
bmi
0.105
steroid_daily_dosage
0.338
number_of_falls
0.143
age
0.0
tscore_ls
0.0
tscore_neck
0.0
tscore_totalHip
0.0
tbs_ls
0.0


In [22]:
from scipy.stats import fisher_exact

# Compute p-values for categorical variables with Fisher exact test
for col in categorical:
    if data[data["any_fracture"] == 0][col].value_counts().shape[0] == 1 \
        or data[data["any_fracture"] == 1][col].value_counts().shape[0] == 1:
        print(f"{col}: Only one value")
        continue
    no_fractures = data[data["any_fracture"] == 0][col]
    no_fx_0, no_fx_1 = no_fractures.value_counts().sort_index()
    any_fractures = data[data["any_fracture"] == 1][col]
    fx_0, fx_1 = any_fractures.value_counts().sort_index()
    table = [[no_fx_0, fx_0], [no_fx_1, fx_1]]
    
    _, pval = fisher_exact(table)
    
    print(f"{col}: {round(pval, 3)}")

hrt: 0.001
hip_fracture_parents: 0.717
osteoporotic_fracture_parents: 0.27
corticosteroids: 0.15
aromatase_inhibitors: 0.008
antiepileptic_drugs: 0.361
rheumatoid_arthritis: 0.038
ankylosing_spondylitis: 0.652
immobility: 0.059
type_1_diabetes: 0.03
copd: 0.911
gastrointestinal_disease: 0.046
early_menopause: 0.185
hyperpara: 0.118
alcohol: 0.063
nicotin: 0.381
decrease_in_height: 0.124
low_back_pain: 0.005
hyperkyphosis: 0.045
falling_test_abnormal: 0.716
Bisphosphonat_prior: 0.0
SERM_prior: 0.271
HRT_prior: 0.77
Denosumab_prior: 0.524
Romosozumab_prior: Only one value
Teriparatide_prior: 0.0
Bisphosphonat_current: 0.0
SERM_current: 0.349
HRT_current: 0.497
Denosumab_current: 1.0
Romosozumab_current: Only one value
Teriparatide_current: 0.505
Bisphosphonat_new: 0.499
SERM_new: 0.188
HRT_new: 0.796
Denosumab_new: 0.064
Romosozumab_new: Only one value
Teriparatide_new: 0.062
No_treatment: 0.0
vertebral_fracture: Only one value
hip_fracture: Only one value
recent_fracture: 0.0
previous_f